In [4]:
import requests
import json
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Groq API key from environment variable
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# === Helper Functions === #
def load_text(file_path):
    """Load text from a file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading text file: {e}")
        return ""

def clean_text(text):
    """Clean the text by removing empty lines."""
    cleaned_text = "\n".join([line for line in text.splitlines() if line.strip() != ""])
    return cleaned_text

def split_text(text, max_tokens=6000):
    """Split the text into smaller chunks based on the max token limit."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) // 4  # Rough estimate of tokens per word
        if current_length + word_length > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = word_length
        else:
            current_chunk.append(word)
            current_length += word_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def query_groq_llm(prompt, context, max_tokens=500):
    """Send a prompt and context to the Groq LLM chat completion API."""
    try:
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }
        payload = {
            "messages": [
                {"role": "system", "content": "You are an assistant helping summarize and query text."},
                {"role": "user", "content": f"{prompt}\n\nContext:\n{context}"}
            ],
            "model": "llama-3.3-70b-versatile",  # Replace with your desired model
            "max_tokens": max_tokens,  # Adjust based on your needs
            "temperature": 0.7  # Adjust randomness
        }
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            return response.json().get("choices", [{}])[0].get("message", {}).get("content", "No response")
        else:
            print(f"Groq API Error: {response.status_code}, {response.text}")
            return f"An error occurred while fetching a response from Groq: {response.status_code}, {response.text}"
    except Exception as e:
        print(f"Error communicating with Groq API: {e}")
        return f"An error occurred while fetching a response from Groq: {e}"

# === Main Functionality === #
def process_text_file(file_path, output_txt_path):
    """Process the text file and generate a clean, compacted educational text file."""
    text = load_text(file_path)
    if not text:
        print("Failed to load text.")
        return

    # Clean the text
    cleaned_text = clean_text(text)

    # Split the text into smaller chunks
    chunks = split_text(cleaned_text, max_tokens=6000)

    # Use a generic query to summarize the entire content
    user_query = "Summarize the key points of the text."

    summarized_text = ""
    for chunk in chunks:
        # Use Groq API for enhanced summarization
        print("\nQuerying Groq LLM for additional insights...")
        llm_response = query_groq_llm(user_query, chunk, max_tokens=500)
        print("\nGroq LLM Response:")
        print(llm_response)
        summarized_text += llm_response + "\n"

    # Save the summarized text to a file
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(summarized_text)

# === Run the System === #
if __name__ == "__main__":
    file_path = "summary_ind.txt"  # Replace with your text file
    output_txt_path = "output_ind.txt"
    process_text_file(file_path, output_txt_path)
    print(f"Summarized text saved to {output_txt_path}")


Querying Groq LLM for additional insights...
Groq API Error: 413, {"error":{"message":"Request too large for model `llama-3.3-70b-versatile` in organization `org_01ja0xazkfekmaw6jd0h73jpjn` on tokens per minute (TPM): Limit 6000, Requested 9930, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}


Groq LLM Response:
An error occurred while fetching a response from Groq: 413, {"error":{"message":"Request too large for model `llama-3.3-70b-versatile` in organization `org_01ja0xazkfekmaw6jd0h73jpjn` on tokens per minute (TPM): Limit 6000, Requested 9930, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}


Querying Groq LLM for additional insights...
Groq API Error: 413, {"error":{"message":"Request too large for model `llama-3.3-70b-versatile` in organization `or